In [1]:
import os
import pandas as pd
from IPython.display import display

pd.options.display.max_columns = None
data = pd.read_csv('../data/raw/games-features.csv')

In [2]:
print(data.describe(include='all'))

              QueryID     ResponseID        QueryName          ResponseName  \
count    13357.000000   13357.000000            13356                 13357   
unique            NaN            NaN            13283                 13199   
top               NaN            NaN  Train Simulator  Train Simulator 2016   
freq              NaN            NaN                5                     9   
mean    353637.916149  353471.814030              NaN                   NaN   
std     146988.380337  147085.351999              NaN                   NaN   
min         10.000000      10.000000              NaN                   NaN   
25%     282860.000000  282640.000000              NaN                   NaN   
50%     374040.000000  373870.000000              NaN                   NaN   
75%     465920.000000  465860.000000              NaN                   NaN   
max     568150.000000  568150.000000              NaN                   NaN   

        ReleaseDate   RequiredAge     DemoCount  De

In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13357 entries, 0 to 13356
Data columns (total 78 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   QueryID                      13357 non-null  int64  
 1   ResponseID                   13357 non-null  int64  
 2   QueryName                    13356 non-null  object 
 3   ResponseName                 13357 non-null  object 
 4   ReleaseDate                  13357 non-null  object 
 5   RequiredAge                  13357 non-null  int64  
 6   DemoCount                    13357 non-null  int64  
 7   DeveloperCount               13357 non-null  int64  
 8   DLCCount                     13357 non-null  int64  
 9   Metacritic                   13357 non-null  int64  
 10  MovieCount                   13357 non-null  int64  
 11  PackageCount                 13357 non-null  int64  
 12  RecommendationCount          13357 non-null  int64  
 13  PublisherCount  

In [4]:
#Confirm that each ResponseID is unique and doesn't map to more than one game
print(data.groupby(by='ResponseID').agg({'ResponseName':'nunique'}).max())

ResponseName    1
dtype: int64


In [5]:
#Drop duplicates based on ResponseID
data.drop_duplicates(subset='ResponseID',inplace=True)
#Drop some fields that won't be helpful
data.drop(columns=['QueryID','QueryName','SupportEmail','SupportURL','PCMinReqsText','PCRecReqsText','LinuxMinReqsText','LinuxRecReqsText','MacMinReqsText','MacRecReqsText'],inplace=True)

In [6]:
#See if games have only 1 genre.  Looks like most have multiple genres
print(data.iloc[:,(data.columns.get_loc('GenreIsNonGame')):(data.columns.get_loc('GenreIsMassivelyMultiplayer')+1)].columns)
data['NumGenres'] = data.iloc[:,(data.columns.get_loc('GenreIsNonGame')):(data.columns.get_loc('GenreIsMassivelyMultiplayer')+1)].sum(axis=1)
print(data['NumGenres'].value_counts())

#See if any NonGame genre items also have other genres
print(data[data['GenreIsNonGame']]['NumGenres'].value_counts())

#Look at the ones that have another genre marked.  Only other genre that is sometimes marked is early access
print(data[(data['GenreIsNonGame']) & (data['NumGenres']==2)].\
      iloc[:,(data.columns.get_loc('GenreIsNonGame')):(data.columns.get_loc('GenreIsMassivelyMultiplayer')+1)])

#Drop non-games.  Explore early access category later, see if it's somehow possible to link existence of early access
#version to later success of full version (if that's how it works).  However this will likely exceed scope of this project
data = data[~data['GenreIsNonGame']]
print(data['GenreIsNonGame'].value_counts())
data.drop(columns='GenreIsNonGame',inplace=True)
print(data.columns)

Index(['GenreIsNonGame', 'GenreIsIndie', 'GenreIsAction', 'GenreIsAdventure',
       'GenreIsCasual', 'GenreIsStrategy', 'GenreIsRPG', 'GenreIsSimulation',
       'GenreIsEarlyAccess', 'GenreIsFreeToPlay', 'GenreIsSports',
       'GenreIsRacing', 'GenreIsMassivelyMultiplayer'],
      dtype='object')
2     3974
3     3253
1     2745
4     1587
0      739
5      615
6      212
7       63
8       17
10       2
9        1
Name: NumGenres, dtype: int64
1    286
2     32
Name: NumGenres, dtype: int64
       GenreIsNonGame  GenreIsIndie  GenreIsAction  GenreIsAdventure  \
4805             True         False          False             False   
5592             True         False          False             False   
5833             True         False          False             False   
6438             True         False          False             False   
7437             True         False          False             False   
8628             True         False          False             False

In [7]:
#Re-check for missing values
print(data.isnull().describe())

       ResponseID ResponseName ReleaseDate RequiredAge DemoCount  \
count       12890        12890       12890       12890     12890   
unique          1            1           1           1         1   
top         False        False       False       False     False   
freq        12890        12890       12890       12890     12890   

       DeveloperCount DLCCount Metacritic MovieCount PackageCount  \
count           12890    12890      12890      12890        12890   
unique              1        1          1          1            1   
top             False    False      False      False        False   
freq            12890    12890      12890      12890        12890   

       RecommendationCount PublisherCount ScreenshotCount SteamSpyOwners  \
count                12890          12890           12890          12890   
unique                   1              1               1              1   
top                  False          False           False          False   
freq     

In [8]:
import numpy as np

#Looks good, just one missing legal notice. Will change from NaN to None after EDA in next workbook,
#since None more compatible with Tableau
#data['LegalNotice'] = data['LegalNotice'].replace({np.nan:None})
print(data[data['LegalNotice'].isnull()]['LegalNotice'])

5326    NaN
Name: LegalNotice, dtype: object


In [9]:
#However, many zeroes are actually missing values. Will replace with np.nan for now, and with None after EDA
columns = ['DeveloperCount','Metacritic','PublisherCount','SteamSpyOwners','SteamSpyOwnersVariance','SteamSpyPlayersEstimate','SteamSpyPlayersVariance']
data[columns] = data[columns].replace(0, np.nan)

In [10]:
#Remove rows without SteamSpy data. I suspect we will primarily focus on owners rather than estimated players since 2009,
#so will delete only where Owners is null, leaving some rows where Players is null
print(data['SteamSpyOwners'].isnull().sum())
print(data['SteamSpyPlayersEstimate'].isnull().sum())
print(data.shape)
data = data[~data['SteamSpyOwners'].isnull()]
print(data.shape)

2596
2838
(12890, 68)
(10294, 68)


In [11]:
#Convert dates to datetime objects
print(data['ReleaseDate'].head(20))
data['ReleaseDate'] = pd.to_datetime(data['ReleaseDate'],errors='coerce',format='%b %d %Y')
print(data['ReleaseDate'].head(20))

0      Nov 1 2000
1      Apr 1 1999
2      May 1 2003
3      Jun 1 2001
4      Nov 1 1999
5      Nov 1 2000
6      Nov 8 1998
7      Mar 1 2004
9      Jun 1 2001
10    Nov 16 2004
11     Nov 1 2004
12     Jun 1 2004
13    Jul 12 2010
14     Nov 1 2004
15    Oct 27 2005
16     May 1 2006
17     Jun 1 2006
18    Oct 10 2007
19    Oct 10 2007
20    Oct 10 2007
Name: ReleaseDate, dtype: object
0    2000-11-01
1    1999-04-01
2    2003-05-01
3    2001-06-01
4    1999-11-01
5    2000-11-01
6    1998-11-08
7    2004-03-01
9    2001-06-01
10   2004-11-16
11   2004-11-01
12   2004-06-01
13   2010-07-12
14   2004-11-01
15   2005-10-27
16   2006-05-01
17   2006-06-01
18   2007-10-10
19   2007-10-10
20   2007-10-10
Name: ReleaseDate, dtype: datetime64[ns]


In [12]:
#Will convert NaTs to None after EDA
#print(data['ReleaseDate'].isnull().sum())
#print(data.loc[data['ReleaseDate'].isnull(),['ReleaseDate']])
#data['ReleaseDate'] = data['ReleaseDate'].astype(object).where(data['ReleaseDate'].notnull(),None)
#print(data.loc[data['ReleaseDate'].isnull(),['ReleaseDate']])

In [13]:
#One-hot encode languages column
data['SupportedLanguages'] = data['SupportedLanguages'].str.replace('*',' ')
data['SupportedLanguages'] = data['SupportedLanguages'].str.replace('Simplified ','Simplified*')
data['SupportedLanguages'] = data['SupportedLanguages'].str.replace('Traditional ','Traditional*')
from collections import Counter
Counter(" ".join(data['SupportedLanguages']).split()).most_common(100)

C:\Users\eagra\AppData\Local\Temp/ipykernel_31056/3879787722.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['SupportedLanguages'] = data['SupportedLanguages'].str.replace('*',' ')


[('English', 10246),
 ('with', 4952),
 ('full', 4952),
 ('audio', 4952),
 ('support', 4951),
 ('languages', 4949),
 ('German', 3387),
 ('French', 3185),
 ('Spanish', 2870),
 ('Italian', 2439),
 ('Russian', 2269),
 ('Japanese', 1150),
 ('Polish', 1123),
 ('Portuguese-Brazil', 954),
 ('Simplified*Chinese', 683),
 ('Dutch', 588),
 ('Korean', 546),
 ('Portuguese', 537),
 ('Czech', 444),
 ('Traditional*Chinese', 416),
 ('Turkish', 339),
 ('Swedish', 305),
 ('Hungarian', 268),
 ('Danish', 211),
 ('Norwegian', 180),
 ('Finnish', 168),
 ('Ukrainian', 156),
 ('Greek', 103),
 ('Romanian', 100),
 ('Arabic', 87),
 ('Thai', 82),
 ('Bulgarian', 80),
 ('[/b]', 5),
 ('English[b]', 3),
 ('Slovakian', 2),
 ('German;', 2),
 ('[/b]languages', 2),
 ('HungarianPolish', 1),
 ('RussianEnglishSpanishFrenchJapaneseCzech', 1),
 ('EnglishRussianSpanishJapaneseCzech', 1),
 ('(all', 1),
 ('support)', 1),
 ('(text', 1),
 ('only)', 1),
 ('[b]', 1),
 ('German[b]', 1),
 ('Spanish[b]', 1),
 ('Russian[b]', 1)]

In [14]:
languages = ['English','German','French','Spanish','Italian','Russian','Japanese','Polish','Portuguese','Chinese',\
             'Dutch','Korean','Czech','Turkish','Swedish','Hungarian','Danish','Norwegian','Finnish','Ukrainian',\
             'Greek','Romanian','Arabic','Thai','Bulgarian','Slovakian']
for l in languages:
    data[l] = data['SupportedLanguages'].str.contains(l)
print(data[languages].head())
data.drop(columns='SupportedLanguages',inplace=True)

   English  German  French  Spanish  Italian  Russian  Japanese  Polish  \
0     True    True    True     True     True    False     False   False   
1     True    True    True     True     True    False     False   False   
2     True    True    True     True     True    False     False   False   
3     True    True    True     True     True    False     False   False   
4     True    True    True    False    False    False     False   False   

   Portuguese  Chinese  Dutch  Korean  Czech  Turkish  Swedish  Hungarian  \
0       False     True  False    True  False    False    False      False   
1       False    False  False   False  False    False    False      False   
2       False    False  False   False  False    False    False      False   
3       False    False  False   False  False    False    False      False   
4       False    False  False    True  False    False    False      False   

   Danish  Norwegian  Finnish  Ukrainian  Greek  Romanian  Arabic   Thai  \
0   False 

In [15]:
#What's the other currency
print(data['PriceCurrency'].unique())
#Do blank currencies only happen when price is 0?
print(data.loc[data['PriceCurrency']==' ']['PriceInitial'].unique())
print(data.loc[data['PriceCurrency']==' ']['PriceFinal'].unique())
#Yes, so this column doesn't add value - let's drop
data.drop(columns='PriceCurrency',inplace=True)

['USD' ' ']
[0.]
[0.]


In [16]:
#save intermediate data file

print(data.shape)

data.to_csv('../data/interim/data_cleaned.csv')

(10294, 92)
